# Code and Documentation
### Part 3 - Models
____
#### GroupD: Nicole Guobadia, Dhruvi Kaswala, Arashpreet Singh Pandher, Karen Angella Edy Setiawan

Note: make sure to shut down all other kernels

In [2]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install()
BiocManager::install("randomForest")
library (randomForest)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.10 (BiocManager 1.30.20), R 3.6.1 (2019-07-05)

Old packages: 'askpass', 'backports', 'BH', 'boot', 'brio', 'broom', 'callr',
  'caret', 'class', 'cli', 'clipr', 'cluster', 'codetools', 'collections',
  'colorspace', 'commonmark', 'crayon', 'curl', 'data.table', 'DBI', 'dbplyr',
  'dichromat', 'diffobj', 'digest', 'dplyr', 'e1071', 'evaluate', 'fansi',
  'forcats', 'foreach', 'formatR', 'fs', 'generics', 'ggplot2', 'glmnet',
  'glue', 'gower', 'gtable', 'gtools', 'haven', 'hexbin', 'highr', 'hms',
  'htmltools', 'htmlwidgets', 'httpuv', 'httr', 'ipred', 'IRdisplay',
  'IRkernel', 'iterators', 'jsonlite', 'KernSmooth', 'knitr', 'labeling',
  'languageserver', 'lars', 'later', 'lattice', 'lava', 'lubridate',
  'magrittr', 'maps', 'markdown', 'MASS', 'Matrix', 'mgcv', 'mime'

In [3]:
install.packages('e1071', dependencies=TRUE)
library('e1071')


  There is a binary version available but the source version is later:
      binary source needs_compilation
e1071  1.7-6 1.7-13              TRUE

  Binaries will be installed
package 'e1071' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\nicol\AppData\Local\Temp\Rtmp8MmqSl\downloaded_packages


Warning message:
"package 'e1071' was built under R version 3.6.3"


In [4]:
set.seed(23)

In [5]:
# load train, test, and gene signature data into notebook
load("data.RData")
load("gene.RData")
ls()

[1] "genesig"       "test_dataset"  "train_dataset"

In [6]:
train_dataset[1:5, 1:5]

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD
GSM1030962_10030.CEL,8.27154,5.02307,7.06640,8.24990,5.75432
GSM1030963_10110.CEL,8.49112,3.95575,6.41582,8.42649,6.77953
GSM1030964_10111.CEL,8.34082,4.68666,6.14716,8.09809,6.35168
GSM1030966_1032.CEL,7.75355,4.97396,6.35464,8.00680,6.79648
GSM1030969_1446.CEL,8.40749,4.55103,6.63393,8.56563,6.34759


In [7]:
test_dataset[1:5, 1:5]

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD
GSM1030965_10138.CEL,8.24190,5.78429,6.65615,8.15517,4.97367
GSM1030967_1113.CEL,7.95457,5.75343,6.89049,7.95516,5.49744
GSM1030968_1132.CEL,7.99231,4.21497,6.28378,8.13866,7.27109
GSM1030979_2120_opnieuw.CEL,8.08617,6.31566,7.07061,8.51588,6.85328
GSM1030982_2252.CEL,7.93179,5.32106,6.42760,8.52095,6.37300


In [8]:
genesig

[1]   283   407   619   680  1738  1780  1885  2025  2303  2304  2374  2651
[13]  2656  2855  3034  3275  3325  3524  3541  3666  3733  3764  3844  3893
[25]  4292  4806  5047  5278  5482  5604  5777  5835  6268  7159  7458  7652
[37]  8075  8129  8457  8493  8602  8737  8827  8887  9338  9789  9794  9877
[49] 10136 10443 10788 10818 11364 11941 11998 12146

##### We train linear SVM model and gives the confusion matrix and accuracy:

In [9]:
# train data with linear svm
train <- train_dataset[, genesig]
train$label <- train_dataset$label
classifier.svm <- svm(label ~ ., data = train, type = 'C-classification', kernel = 'linear')

In [10]:
# predict using test data
pred.svm <- predict(classifier.svm, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.svm, test_dataset$label)
conf_matrix

        
pred.svm   0   1
       0 507   4
       1   7 547

In [11]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9896714

##### We train logistic regression model and gives the confusion matrix and accuracy:

In [12]:
# train data with log reg
classifier.logreg <- glm(label ~ ., data = train,family = binomial)

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"


In [13]:
# predict using test data
pred.logreg <- predict(classifier.logreg, test_dataset[,1:length(test_dataset)-1], type = "response")
pred.logreg <- round(pred.logreg)
conf_matrix <- table(pred.logreg, test_dataset$label)
conf_matrix

           
pred.logreg   0   1
          0 508   5
          1   6 546

In [14]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9896714

##### We train random forest model and gives the confusion matrix and accuracy:

In [15]:
# randomforest classifier
classifier.tree <- randomForest(as.factor(train$label) ~ ., data=train)

In [16]:
pred.tree <- predict (classifier.tree, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.tree, test_dataset$label)
conf_matrix

         
pred.tree   0   1
        0 511  10
        1   3 541

In [17]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9877934

##### Build ensemble with majority voting:

In [18]:
#The majority vote
# reference https://www.analyticsvidhya.com/blog/2017/02/introduction-to-ensembling-along-with-implementation-in-r/
testSet <- as.factor(ifelse(pred.tree == '1' & pred.logreg == '1', '1', ifelse(pred.tree
                                                                        == '1' & pred.svm == '1','1', ifelse(pred.logreg == '1' & pred.svm == '1', '1', '0'))))

In [19]:
conf_matrix <- table(testSet, test_dataset$label)
conf_matrix

       
testSet   0   1
      0 509   5
      1   5 546

In [20]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9906103

### Alternate gene signature comparison from other team (Genome Seeker)

##### We train all the models again with new gene signatures:

In [21]:
alt.genesig <- colnames(read.table("alternate_genesig.txt", sep=",", header=T))
alt.genesig

[1] "IGHA2"     "SOCS2"     "IGKC"      "KIT"       "PRAME"     "IGLJ2"    
 [7] "PPBP"      "ALDH1A1"   "PDGFC"     "CLC"       "IGF2BP2"   "EPS8"     
[13] "HOXB6"     "PF4"       "G0S2"      "HOXA9"     "SECISBP2L" "DEFA1"    
[19] "HDC"       "PTH2R"     "FGF13"     "CST3"      "RORA"      "CXCL3"    
[25] "DPY19L1P1" "PAX5"      "ADAM28"    "DNM1"      "FPR1"      "NR4A3"    
[31] "TFPI"      "TOP1"      "LEF1"      "TST"       "SLCO3A1"   "CTSW"     
[37] "PRSS21"    "TIMP1"     "CYP4F3"    "TMEM243"   "ALOX12"    "ABL1"     
[43] "PROS1"     "LRMP"      "DISC1"     "MYH10"     "RNF216"    "FAN1"     
[49] "HOMER3"

In [22]:
alt.genesig.ind <- c(1:49)
for (j in 1:49){
    alt.genesig.ind[j] <- which(colnames(train_dataset) == alt.genesig[j])
}
alt.genesig.ind[1:5]

[1]  8827  1738 12198  2855  2212

In [23]:
# Train models
train <- train_dataset[, alt.genesig.ind]
train$label <- train_dataset$label
classifier.svm <- svm(label ~ ., data = train, type = 'C-classification', kernel = 'linear')
classifier.logreg <- glm(label ~ ., data = train,family = binomial)
classifier.tree <- randomForest(as.factor(train$label) ~ ., data=train)

In [24]:
# SVM prediction
pred.alt.svm <- predict(classifier.svm, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.alt.svm, test_dataset$label)
conf_matrix

            
pred.alt.svm   0   1
           0 507  14
           1   7 537

In [25]:
alt.accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
alt.accuracy

[1] 0.9802817

In [26]:
# Logistic regression prediction
pred.alt.logreg <- predict(classifier.logreg, test_dataset[,1:length(test_dataset)-1], type = "response")
pred.alt.logreg <- round(pred.alt.logreg)
conf_matrix <- table(pred.alt.logreg, test_dataset$label)
conf_matrix

               
pred.alt.logreg   0   1
              0 508  15
              1   6 536

In [27]:
alt.accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
alt.accuracy

[1] 0.9802817

In [28]:
# Random forest prediction
pred.alt.tree <- predict (classifier.tree, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.alt.tree, test_dataset$label)
conf_matrix

             
pred.alt.tree   0   1
            0 509  14
            1   5 537

In [29]:
alt.accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
alt.accuracy

[1] 0.9821596

##### Build ensemble with majority voting:

In [30]:
#The majority vote
# reference https://www.analyticsvidhya.com/blog/2017/02/introduction-to-ensembling-along-with-implementation-in-r/
testSet <- as.factor(ifelse(pred.alt.tree == '1' & pred.alt.logreg == '1', '1', ifelse(pred.alt.tree
                                                                        == '1' & pred.alt.svm == '1','1', ifelse(pred.alt.logreg == '1' & pred.alt.svm == '1', '1', '0'))))

In [31]:
conf_matrix <- table(testSet, test_dataset$label)
conf_matrix

       
testSet   0   1
      0 507  13
      1   7 538

In [32]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9812207

## Cross validation on dataset 1

In [33]:
load("AML_datasets.RData")

In [44]:
# gene expression data
dim(data.1)
data.1[1:5, 1:8]

[1] 12708  2500

,AHW_ALL_BA_025919_13_A_PZ.CEL,AHW_ALL_BA_032751_71_A_PZ.CEL,AHW_ALL_BN_022495_7_A_PZ.CEL,AHW_ALL_MLL_031351_13_A_PZ.CEL,AHW_ALL_MLL_036187_80_A_PZ.CEL,AHW_ALL_Ph_030031_32_A_PZ.CEL,AHW_ALL_Ph_032683_15_A_PZ.CEL,AHW_ALL_Ph_035655_63_A_PZ.CEL
PAX8,9.221573,8.524435,8.606060,8.841501,8.882826,9.011544,8.975590,8.940409
CCL5,7.990144,6.648112,6.382602,6.469583,8.332415,6.830385,8.753525,6.370298
MMP14,8.391732,8.183514,7.909023,8.105744,7.820992,7.859977,8.128827,7.956449
DTX2P1-UPK3BP1-PMS2P11,9.280415,8.798847,8.948909,8.941517,8.725958,8.975915,9.130401,8.997386
BAD,5.579486,5.934621,5.646312,5.899868,5.651731,5.798776,5.849792,6.002240


In [45]:
# subset of data containing only AML and ALL samples
leukemia <- c('AML', 'ALL')
subset.info <- subset(info.1, Disease %in% leukemia)

In [46]:
# phenotype data (AML and ALL only)
dim(subset.info)
subset.info[1:5,]

[1] 1942    8

,Dataset,GSE,Condition,Disease,Tissue,FAB,Filename,FAB_all
AHW_ALL_BA_025919_13_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_BA_025919_13_A_PZ.CEL,unknown
AHW_ALL_BA_032751_71_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_BA_032751_71_A_PZ.CEL,unknown
AHW_ALL_BN_022495_7_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_BN_022495_7_A_PZ.CEL,unknown
AHW_ALL_MLL_031351_13_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_MLL_031351_13_A_PZ.CEL,unknown
AHW_ALL_MLL_036187_80_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_MLL_036187_80_A_PZ.CEL,unknown


In [53]:
# gene expression data (AML and ALL only)
subset.data.1 <- data.1[,rownames(subset.info)]
dim(subset.data.1)
subset.data.1[1:10,1:10]

[1] 12708  1942

,AHW_ALL_BA_025919_13_A_PZ.CEL,AHW_ALL_BA_032751_71_A_PZ.CEL,AHW_ALL_BN_022495_7_A_PZ.CEL,AHW_ALL_MLL_031351_13_A_PZ.CEL,AHW_ALL_MLL_036187_80_A_PZ.CEL,AHW_ALL_Ph_030031_32_A_PZ.CEL,AHW_ALL_Ph_032683_15_A_PZ.CEL,AHW_ALL_Ph_035655_63_A_PZ.CEL,AHW_ALL_TA_025336_26_A_PZ.CEL,AHW_ALL_TA_032725_16_A_PZ.CEL
PAX8,9.221573,8.524435,8.606060,8.841501,8.882826,9.011544,8.975590,8.940409,9.342263,8.966156
CCL5,7.990144,6.648112,6.382602,6.469583,8.332415,6.830385,8.753525,6.370298,6.971186,7.302477
MMP14,8.391732,8.183514,7.909023,8.105744,7.820992,7.859977,8.128827,7.956449,8.285775,8.335117
DTX2P1-UPK3BP1-PMS2P11,9.280415,8.798847,8.948909,8.941517,8.725958,8.975915,9.130401,8.997386,9.294638,9.094800
BAD,5.579486,5.934621,5.646312,5.899868,5.651731,5.798776,5.849792,6.002240,5.642667,5.768286
PRPF8,9.587992,10.605028,10.367226,10.493261,10.684440,9.850415,10.068460,10.259652,10.168444,9.143968
CAPNS1,10.143771,10.613034,9.836321,10.701126,10.524360,10.115055,10.252378,10.722423,9.809753,9.503368
RPL35,11.364570,11.664621,12.081325,12.029874,11.856132,11.909563,11.671361,12.208231,10.956313,11.262788
EIF4G2,10.956290,11.632159,11.690962,11.328697,11.388613,11.197291,11.070642,11.471852,11.881013,11.051533
EIF3D,10.026995,11.412836,10.966108,11.176077,11.117741,10.518100,11.040760,11.182870,10.474084,10.038692


In [54]:
subset.data.1 <- as.data.frame(t(subset.data.1))
dim(subset.data.1)
subset.data.1[1:10,1:10]

[1]  1942 12708

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD,PRPF8,CAPNS1,RPL35,EIF4G2,EIF3D
AHW_ALL_BA_025919_13_A_PZ.CEL,9.221573,7.990144,8.391732,9.280415,5.579486,9.587992,10.143771,11.36457,10.95629,10.02699
AHW_ALL_BA_032751_71_A_PZ.CEL,8.524435,6.648112,8.183514,8.798847,5.934621,10.605028,10.613034,11.66462,11.63216,11.41284
AHW_ALL_BN_022495_7_A_PZ.CEL,8.606060,6.382602,7.909023,8.948909,5.646312,10.367226,9.836321,12.08132,11.69096,10.96611
AHW_ALL_MLL_031351_13_A_PZ.CEL,8.841501,6.469583,8.105744,8.941517,5.899868,10.493261,10.701126,12.02987,11.32870,11.17608
AHW_ALL_MLL_036187_80_A_PZ.CEL,8.882826,8.332415,7.820992,8.725958,5.651731,10.684440,10.524360,11.85613,11.38861,11.11774
AHW_ALL_Ph_030031_32_A_PZ.CEL,9.011544,6.830385,7.859977,8.975915,5.798776,9.850415,10.115055,11.90956,11.19729,10.51810
AHW_ALL_Ph_032683_15_A_PZ.CEL,8.975590,8.753525,8.128827,9.130401,5.849792,10.068460,10.252378,11.67136,11.07064,11.04076
AHW_ALL_Ph_035655_63_A_PZ.CEL,8.940409,6.370298,7.956449,8.997386,6.002240,10.259652,10.722423,12.20823,11.47185,11.18287
AHW_ALL_TA_025336_26_A_PZ.CEL,9.342263,6.971186,8.285775,9.294638,5.642667,10.168444,9.809753,10.95631,11.88101,10.47408
AHW_ALL_TA_032725_16_A_PZ.CEL,8.966156,7.302477,8.335117,9.094800,5.768286,9.143968,9.503368,11.26279,11.05153,10.03869


In [55]:
# add class label
subset.data.1$label <- with(subset.info, ifelse(Disease=='AML', 1, 0))
test.dataset.1 <- subset.data.1
test.dataset.1$label[1:10]
dim(test.dataset.1)

[1] 0 0 0 0 0 0 0 0 0 0

[1]  1942 12709

### Linear SVM cross validation

In [56]:
# predict using test data
pred.svm <- predict(classifier.svm, test.dataset.1[,1:length(test.dataset.1)-1], decision.values = TRUE)
conf_matrix <- table(pred.svm, test.dataset.1$label)
conf_matrix

        
pred.svm   0   1
       0 924   9
       1  32 977

In [57]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9788877

### Logistic regression cross validation

In [59]:
# predict using test data
pred.logreg <- predict(classifier.logreg, test.dataset.1[,1:length(test_dataset)-1], type = "response")
pred.logreg <- round(pred.logreg)
conf_matrix <- table(pred.logreg, test.dataset.1$label)
conf_matrix

           
pred.logreg   0   1
          0 933   8
          1  23 978

In [60]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9840371

### Decision tree cross validation

In [61]:
pred.tree <- predict (classifier.tree, test.dataset.1[,1:length(test.dataset.1)-1], decision.values = TRUE)
conf_matrix <- table(pred.tree, test.dataset.1$label)
conf_matrix

         
pred.tree   0   1
        0 928   5
        1  28 981

In [62]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9830072

### Ensemble cross validation

In [63]:
testSet <- as.factor(ifelse(pred.tree == '1' & pred.logreg == '1', '1', ifelse(pred.tree
                                                                        == '1' & pred.svm == '1','1', ifelse(pred.logreg == '1' & pred.svm == '1', '1', '0'))))
conf_matrix <- table(testSet, test.dataset.1$label)
conf_matrix

       
testSet   0   1
      0 929   8
      1  27 978

In [64]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9819773